In [1]:
import numpy as np 
import pandas as pd 
from subprocess import check_output
#print(check_output(["ls", "../input/"]))
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from os.path import join as opj
from matplotlib import pyplot as plt
import tensorflow as tf
import os
%matplotlib inline


In [2]:
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation,Add, ZeroPadding2D ,AveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers,regularizers
from keras.optimizers import Adam,SGD
from keras.initializers import glorot_uniform
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
import keras.backend as K
K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [8]:
train = pd.read_json("input/train.json")
#test = pd.read_json("input/test.json")

In [9]:
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)
target_train=train['is_iceberg']

In [5]:
X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis], X_band_test_2[:, :, :, np.newaxis], ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [10]:

X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=2,stratify = target_train, train_size=0.80)

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### Cross Validation Folds

In [42]:
def data_augmentation(X_input,Y_input,batch_size = 32):
    data_aug = ImageDataGenerator(#featurewise_center=True,
                             #featurewise_std_normalization=True,
                             #zca_whitening=True,
                             rotation_range=0,
                             width_shift_range = 0,
                             height_shift_range = 0,
                             zoom_range = 0,
                             data_format = 'channels_last',
                             horizontal_flip = True,
                             vertical_flip = True)#,fill_mode = 'constant',cval = 0)
    data_aug_batches = data_aug.flow(X_input,Y_input,batch_size = batch_size)
    return data_aug_batches


In [20]:
def basic_cbamd_layer(X_input,filter_num =64,kernel_size = (3,3),conv_padding = 'valid',pool_size =(2,2),pool_strides = (2,2),pool_padding = 'valid',dropout_rate = 0.2):
    X = Conv2D(filter_num, kernel_size, padding = conv_padding)(X_input)
    #X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size,strides = pool_strides,padding = pool_padding)(X)
    X = Dropout(rate = dropout_rate)(X)
    return X

def dense_layer(X_input, units = 1,activation = 'relu',dropout_rate = 0.2):
    X = Dense(units = units,activation = activation)(X_input)
    X = Dropout(rate = dropout_rate)(X)
    return X
#kernel_regularizer=regularizers.l2(0.01)
def cnnmodel(input_shape,lr = 0.0001):
    X_input = Input(input_shape)
    #First composite layer
    X = basic_cbamd_layer(X_input,
                           filter_num =64,
                           kernel_size = (5,5),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Second composite layer
    X = basic_cbamd_layer(X,
                           filter_num =128,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Third composite layer
    X = basic_cbamd_layer(X,
                           filter_num =128,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(2,2),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Forth composite layer
    X = basic_cbamd_layer(X,
                           filter_num =64,
                           kernel_size = (3,3),
                           conv_padding = 'valid',
                           pool_size =(3,3),
                           pool_strides = (2,2),
                           pool_padding = 'valid',
                           dropout_rate = 0.2)
    #Flatten layer
    X = Flatten()(X)
    #First dense layer
    X = dense_layer(X, units = 256,activation = 'relu',dropout_rate = 0.2)
    #Second dense layer
    X = dense_layer(X, units = 128,activation = 'relu',dropout_rate = 0.2)
    #Decision layer
    X = dense_layer(X, units = 1,activation = 'sigmoid',dropout_rate = 0)
    
    model = Model(inputs=X_input,outputs=X)
    optimizer=Adam(lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    #optimizer = SGD(lr, decay=0, momentum=0.9, nesterov=True)
    model.compile(loss='binary_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    model.summary()
    return model

def get_callbacks(filepath = ".model_weights.hdf5", patience=7):
    #es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [msave]

#callbacks = get_callbacks(filepath=file_path, patience=3)

In [23]:
Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
def fitmodel(model,X_train,y_train,X_valid,y_valid,augment = False,epochs = 50,batch_size = 32,filepath = ".model_weights.hdf5"):
    if augment == False:
        result = model.fit(X_train, y_train,
                          batch_size=batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(X_valid, y_valid),
                          callbacks=get_callbacks(filepath))
    else:
        result = model.fit_generator(data_augmentation(X_train,y_train,batch_size = batch_size),
                          steps_per_epoch = len(X_train_cv)/batch_size,
                          epochs=epochs,
                          verbose=1,
                          validation_data=(X_valid, y_valid),
                          callbacks=get_callbacks(filepath))
    return result

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_9 (Activation)    (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_10 (Activation)   (None, 33, 33, 128)       0         
__________

In [39]:
for i,layer in enumerate(Cnnmodel.layers):
    print(i,layer.name)

0 input_3
1 conv2d_9
2 activation_9
3 max_pooling2d_9
4 dropout_15
5 conv2d_10
6 activation_10
7 max_pooling2d_10
8 dropout_16
9 conv2d_11
10 activation_11
11 max_pooling2d_11
12 dropout_17
13 conv2d_12
14 activation_12
15 max_pooling2d_12
16 dropout_18
17 flatten_3
18 dense_7
19 dropout_19
20 dense_8
21 dropout_20
22 dense_9
23 dropout_21


In [45]:
Cnnmodel.load_weights('model save/Model 1-Advanced CNN with DA and CV5/2018.1.1_local0.2069OF.hdf5')
for layer in Cnnmodel.layers[:14]:
    layer.trainable = False
Cnnmodel.summary()
result_Cnn = fitmodel(Cnnmodel,X_train_cv,y_train_cv,X_valid,y_valid,augment = True,epochs = 200,batch_size = 32,filepath = 'model save/Model 1-Advanced CNN with DA and CV5/2018.1.1.hdf5')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_9 (Activation)    (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_10 (Activation)   (None, 33, 33, 128)       0         
__________

c:\users\yltbe\anaconda3\envs\tensorflow_gpu\lib\site-packages\keras\engine\training.py:973: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


41/40 [==============================] - 6s 142ms/step - loss: 0.2740 - acc: 0.8688 - val_loss: 0.2555 - val_acc: 0.9003
Epoch 2/200
41/40 [==============================] - 5s 133ms/step - loss: 0.2831 - acc: 0.8677 - val_loss: 0.2313 - val_acc: 0.8972
Epoch 3/200
41/40 [==============================] - 5s 134ms/step - loss: 0.2551 - acc: 0.8970 - val_loss: 0.2525 - val_acc: 0.8910
Epoch 4/200
41/40 [==============================] - 5s 133ms/step - loss: 0.2875 - acc: 0.8578 - val_loss: 0.2784 - val_acc: 0.8629
Epoch 5/200
41/40 [==============================] - 5s 132ms/step - loss: 0.2568 - acc: 0.8902 - val_loss: 0.2429 - val_acc: 0.9003
Epoch 6/200
41/40 [==============================] - 5s 133ms/step - loss: 0.2283 - acc: 0.8925 - val_loss: 0.2782 - val_acc: 0.8629
Epoch 7/200
41/40 [==============================] - 5s 132ms/step - loss: 0.2432 - acc: 0.8960 - val_loss: 0.2509 - val_acc: 0.8785
Epoch 8/200
41/40 [==============================] - 5s 134ms/step - loss: 0.2301

41/40 [==============================] - 5s 133ms/step - loss: 0.1759 - acc: 0.9161 - val_loss: 0.2197 - val_acc: 0.9128
Epoch 63/200
41/40 [==============================] - 5s 131ms/step - loss: 0.1697 - acc: 0.9306 - val_loss: 0.2115 - val_acc: 0.9221
Epoch 64/200
41/40 [==============================] - 5s 131ms/step - loss: 0.1757 - acc: 0.9237 - val_loss: 0.2269 - val_acc: 0.9003
Epoch 65/200
41/40 [==============================] - 5s 134ms/step - loss: 0.1687 - acc: 0.9321 - val_loss: 0.2113 - val_acc: 0.9159
Epoch 66/200
41/40 [==============================] - 6s 136ms/step - loss: 0.1709 - acc: 0.9298 - val_loss: 0.2283 - val_acc: 0.9065
Epoch 67/200
41/40 [==============================] - 5s 134ms/step - loss: 0.1843 - acc: 0.9115 - val_loss: 0.2236 - val_acc: 0.9097
Epoch 68/200
41/40 [==============================] - 5s 133ms/step - loss: 0.1868 - acc: 0.9165 - val_loss: 0.2124 - val_acc: 0.9190
Epoch 69/200
41/40 [==============================] - 5s 131ms/step - loss:

KeyboardInterrupt: 

In [44]:
Cnnmodel.load_weights('model save/Model 1-Advanced CNN with DA and CV5/2018.1.1.hdf5')
Cnnmodel.evaluate(X_valid, y_valid)
#Cnnmodel.evaluate(X_train_cv, y_train_cv)

321/321 [==============================] - 0s 1ms/step


[0.18129999453858855, 0.94392523364485981]

In [149]:
def save_history(result,name = 'fit_history.csv',path = os.getcwd()):
    train_loss = result.history['loss']
    val_loss = result.history['val_loss']
    train_accuracy = result.history['acc']
    val_accuracy = result.history['val_acc']
    fit_history = pd.DataFrame({'train_loss':train_loss,'val_loss':val_loss,'train_accuracy':train_accuracy,'val_accuracy':val_accuracy})
    fit_history.to_csv(os.path.join(path,name))

In [137]:
save_history(result_Cnn)

NameError: name 'result_Cnn' is not defined

In [151]:
Train_cv = StratifiedKFold(n_splits=5,shuffle = True,random_state=4).split(X_train,target_train)
cv_val_list = []
for i,masks in enumerate(Train_cv):
    train_mask,valid_mask = masks
    X_train_cv = X_train[train_mask]
    y_train_cv = target_train[train_mask]
    X_valid = X_train[valid_mask]
    y_valid = target_train[valid_mask]
    path = 'model save/Model 1-Advanced CNN with DA and CV5/'
    modelseries = '.cv'+str(i+1)+'.hdf5'
    modelpath = os.path.join(path,modelseries)
    Cnnmodel=cnnmodel(input_shape = (75,75,3),lr = 0.0001)
    result_Cnn = fitmodel(Cnnmodel,X_train_cv,y_train_cv,X_valid,y_valid,augment = True,epochs = 150,batch_size = 32,filepath = modelpath)
    historyseries = 'cv'+str(i+1)+'.csv'
    save_history(result_Cnn,name = historyseries,path = path)
    Cnnmodel.load_weights(filepath=modelpath)
    score = Cnnmodel.evaluate(X_valid, y_valid, verbose=1)
    cv_val_list.append(score)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 75, 75, 3)         0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 71, 71, 64)        4864      
_________________________________________________________________
activation_33 (Activation)   (None, 71, 71, 64)        0         
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_57 (Dropout)         (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 33, 33, 128)       73856     
_________________________________________________________________
activation_34 (Activation)   (None, 33, 33, 128)       0         
__________

41/40 [==============================] - 5s 129ms/step - loss: 0.3026 - acc: 0.8665 - val_loss: 0.3925 - val_acc: 0.8292
Epoch 37/150
41/40 [==============================] - 5s 131ms/step - loss: 0.3642 - acc: 0.8152 - val_loss: 0.3348 - val_acc: 0.8509
Epoch 38/150
41/40 [==============================] - 5s 131ms/step - loss: 0.3224 - acc: 0.8404 - val_loss: 0.3338 - val_acc: 0.8354
Epoch 39/150
41/40 [==============================] - 5s 130ms/step - loss: 0.2998 - acc: 0.8536 - val_loss: 0.3116 - val_acc: 0.8696
Epoch 40/150
41/40 [==============================] - 5s 131ms/step - loss: 0.3055 - acc: 0.8673 - val_loss: 0.3641 - val_acc: 0.8385
Epoch 41/150
41/40 [==============================] - 5s 130ms/step - loss: 0.3012 - acc: 0.8566 - val_loss: 0.3435 - val_acc: 0.8354
Epoch 42/150
41/40 [==============================] - 5s 132ms/step - loss: 0.3143 - acc: 0.8543 - val_loss: 0.3079 - val_acc: 0.8789
Epoch 43/150
41/40 [==============================] - 5s 131ms/step - loss:

Epoch 1/150
41/40 [==============================] - 10s 244ms/step - loss: 1.2527 - acc: 0.4994 - val_loss: 0.6792 - val_acc: 0.5358
Epoch 2/150
41/40 [==============================] - 5s 134ms/step - loss: 0.7273 - acc: 0.5364 - val_loss: 0.6434 - val_acc: 0.6417
Epoch 3/150
41/40 [==============================] - 6s 138ms/step - loss: 0.6720 - acc: 0.5636 - val_loss: 0.6135 - val_acc: 0.5919
Epoch 4/150
41/40 [==============================] - 5s 133ms/step - loss: 0.6335 - acc: 0.5741 - val_loss: 0.5777 - val_acc: 0.6324
Epoch 5/150
41/40 [==============================] - 5s 134ms/step - loss: 0.5992 - acc: 0.6154 - val_loss: 0.5623 - val_acc: 0.6604
Epoch 6/150
41/40 [==============================] - 5s 134ms/step - loss: 0.5692 - acc: 0.6568 - val_loss: 0.5554 - val_acc: 0.6916
Epoch 7/150
41/40 [==============================] - 6s 134ms/step - loss: 0.5670 - acc: 0.6744 - val_loss: 0.5446 - val_acc: 0.6854
Epoch 8/150
41/40 [==============================] - 6s 136ms/step -

41/40 [==============================] - 6s 138ms/step - loss: 0.2190 - acc: 0.8982 - val_loss: 0.2453 - val_acc: 0.8941
Epoch 124/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2209 - acc: 0.9039 - val_loss: 0.2386 - val_acc: 0.8879
Epoch 125/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2266 - acc: 0.9009 - val_loss: 0.2339 - val_acc: 0.8879
Epoch 126/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2243 - acc: 0.9013 - val_loss: 0.2307 - val_acc: 0.8879
Epoch 127/150
41/40 [==============================] - 5s 132ms/step - loss: 0.2038 - acc: 0.9074 - val_loss: 0.2278 - val_acc: 0.8754
Epoch 128/150
41/40 [==============================] - 5s 130ms/step - loss: 0.1959 - acc: 0.9169 - val_loss: 0.2844 - val_acc: 0.8629
Epoch 129/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2116 - acc: 0.9085 - val_loss: 0.2433 - val_acc: 0.8754
Epoch 130/150
41/40 [==============================] - 5s 131ms/step 

41/40 [==============================] - 6s 136ms/step - loss: 0.2935 - acc: 0.8502 - val_loss: 0.2585 - val_acc: 0.8723
Epoch 69/150
41/40 [==============================] - 6s 134ms/step - loss: 0.2545 - acc: 0.8925 - val_loss: 0.2705 - val_acc: 0.8816
Epoch 70/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2280 - acc: 0.8925 - val_loss: 0.2668 - val_acc: 0.8816
Epoch 71/150
41/40 [==============================] - 6s 135ms/step - loss: 0.2522 - acc: 0.8876 - val_loss: 0.2661 - val_acc: 0.8723
Epoch 72/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2419 - acc: 0.8970 - val_loss: 0.2582 - val_acc: 0.8816
Epoch 73/150
41/40 [==============================] - 5s 132ms/step - loss: 0.2503 - acc: 0.8815 - val_loss: 0.2835 - val_acc: 0.8598
Epoch 74/150
41/40 [==============================] - 5s 134ms/step - loss: 0.2277 - acc: 0.9031 - val_loss: 0.2788 - val_acc: 0.8879
Epoch 75/150
41/40 [==============================] - 6s 136ms/step - loss:

41/40 [==============================] - 5s 133ms/step - loss: 0.1949 - acc: 0.9154 - val_loss: 0.2424 - val_acc: 0.8754
Epoch 130/150
41/40 [==============================] - 6s 141ms/step - loss: 0.1966 - acc: 0.9131 - val_loss: 0.2237 - val_acc: 0.9128
Epoch 131/150
41/40 [==============================] - 6s 136ms/step - loss: 0.1867 - acc: 0.9123 - val_loss: 0.2675 - val_acc: 0.8754
Epoch 132/150
41/40 [==============================] - 6s 137ms/step - loss: 0.1933 - acc: 0.9154 - val_loss: 0.2498 - val_acc: 0.8879
Epoch 133/150
41/40 [==============================] - 5s 132ms/step - loss: 0.2424 - acc: 0.8888 - val_loss: 0.2775 - val_acc: 0.8660
Epoch 134/150
41/40 [==============================] - 5s 134ms/step - loss: 0.2164 - acc: 0.9036 - val_loss: 0.2488 - val_acc: 0.8692
Epoch 135/150
41/40 [==============================] - 5s 132ms/step - loss: 0.2131 - acc: 0.8986 - val_loss: 0.2415 - val_acc: 0.8972
Epoch 136/150
41/40 [==============================] - 5s 133ms/step 

41/40 [==============================] - 5s 133ms/step - loss: 0.2565 - acc: 0.8887 - val_loss: 0.2757 - val_acc: 0.8781
Epoch 75/150
41/40 [==============================] - 5s 132ms/step - loss: 0.2595 - acc: 0.8743 - val_loss: 0.2684 - val_acc: 0.9000
Epoch 76/150
41/40 [==============================] - 6s 143ms/step - loss: 0.2398 - acc: 0.8879 - val_loss: 0.3211 - val_acc: 0.8406
Epoch 77/150
41/40 [==============================] - 6s 138ms/step - loss: 0.2497 - acc: 0.8819 - val_loss: 0.2866 - val_acc: 0.8844
Epoch 78/150
41/40 [==============================] - 5s 132ms/step - loss: 0.2430 - acc: 0.8835 - val_loss: 0.2645 - val_acc: 0.9062
Epoch 79/150
41/40 [==============================] - 6s 136ms/step - loss: 0.2277 - acc: 0.8948 - val_loss: 0.2722 - val_acc: 0.8938
Epoch 80/150
41/40 [==============================] - 5s 133ms/step - loss: 0.2406 - acc: 0.8826 - val_loss: 0.2640 - val_acc: 0.9000
Epoch 81/150
41/40 [==============================] - 6s 145ms/step - loss:

Epoch 135/150
41/40 [==============================] - 5s 131ms/step - loss: 0.1856 - acc: 0.9138 - val_loss: 0.2599 - val_acc: 0.9094
Epoch 136/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2071 - acc: 0.9093 - val_loss: 0.2464 - val_acc: 0.9125
Epoch 137/150
41/40 [==============================] - 5s 132ms/step - loss: 0.1799 - acc: 0.9169 - val_loss: 0.2719 - val_acc: 0.9094
Epoch 138/150
41/40 [==============================] - 5s 133ms/step - loss: 0.2072 - acc: 0.9079 - val_loss: 0.2557 - val_acc: 0.8969
Epoch 139/150
41/40 [==============================] - 5s 133ms/step - loss: 0.1964 - acc: 0.9154 - val_loss: 0.2435 - val_acc: 0.9219
Epoch 140/150
41/40 [==============================] - 5s 130ms/step - loss: 0.1929 - acc: 0.9185 - val_loss: 0.2616 - val_acc: 0.9031
Epoch 141/150
41/40 [==============================] - 5s 134ms/step - loss: 0.1929 - acc: 0.9223 - val_loss: 0.2579 - val_acc: 0.9125
Epoch 142/150
41/40 [==============================] - 

41/40 [==============================] - 5s 131ms/step - loss: 0.2498 - acc: 0.8902 - val_loss: 0.2525 - val_acc: 0.9094
Epoch 81/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2336 - acc: 0.8909 - val_loss: 0.2493 - val_acc: 0.8750
Epoch 82/150
41/40 [==============================] - 6s 138ms/step - loss: 0.2573 - acc: 0.8857 - val_loss: 0.2447 - val_acc: 0.8844
Epoch 83/150
41/40 [==============================] - 6s 136ms/step - loss: 0.2487 - acc: 0.8879 - val_loss: 0.2378 - val_acc: 0.9219
Epoch 84/150
41/40 [==============================] - 6s 135ms/step - loss: 0.2358 - acc: 0.8948 - val_loss: 0.2495 - val_acc: 0.9156
Epoch 85/150
41/40 [==============================] - 5s 131ms/step - loss: 0.2404 - acc: 0.8909 - val_loss: 0.2689 - val_acc: 0.8906
Epoch 86/150
41/40 [==============================] - 6s 143ms/step - loss: 0.2343 - acc: 0.8971 - val_loss: 0.2404 - val_acc: 0.9156
Epoch 87/150
41/40 [==============================] - 6s 141ms/step - loss:

Epoch 141/150
41/40 [==============================] - 5s 131ms/step - loss: 0.1866 - acc: 0.9253 - val_loss: 0.2446 - val_acc: 0.9281
Epoch 142/150
41/40 [==============================] - 5s 132ms/step - loss: 0.2039 - acc: 0.9054 - val_loss: 0.2434 - val_acc: 0.8844
Epoch 143/150
41/40 [==============================] - 5s 131ms/step - loss: 0.1664 - acc: 0.9268 - val_loss: 0.2475 - val_acc: 0.9156
Epoch 144/150
41/40 [==============================] - 5s 131ms/step - loss: 0.1843 - acc: 0.9077 - val_loss: 0.2761 - val_acc: 0.9187
Epoch 145/150
41/40 [==============================] - 6s 135ms/step - loss: 0.1880 - acc: 0.9086 - val_loss: 0.2392 - val_acc: 0.8844
Epoch 146/150
41/40 [==============================] - 5s 131ms/step - loss: 0.1811 - acc: 0.9176 - val_loss: 0.2517 - val_acc: 0.9062
Epoch 147/150
41/40 [==============================] - 5s 132ms/step - loss: 0.1899 - acc: 0.9215 - val_loss: 0.2373 - val_acc: 0.9062
Epoch 148/150
41/40 [==============================] - 

In [152]:
cv_val_list

[[0.22919706774728013, 0.92236024844720499],
 [0.20885347976491458, 0.88473520249221183],
 [0.22079534935431316, 0.91277258566978192],
 [0.2388879343867302, 0.90625],
 [0.22113464772701263, 0.90937500000000004]]

### Prediction

In [40]:
path = 'prediction/model1+two 0.17/'
model_list = os.listdir(path)

In [42]:
prediction = pd.DataFrame()
prediction['id']=test['id']
for name in model_list:
    Cnnmodel.load_weights(os.path.join(path,name))
    pred = Cnnmodel.predict(X_test)
    prediction[name] = np.squeeze(pred)

In [119]:
prediction['is_iceberg'] = np.where(np.all(prediction[prediction.columns[1:]]>0.9,axis = 1),
                                    np.max(prediction[prediction.columns[1:]],axis = 1),
                                     np.where(np.all(prediction[prediction.columns[1:]]<0.1,axis = 1),np.min(prediction[prediction.columns[1:]],axis = 1),np.median(prediction[prediction.columns[1:]],axis = 1)))

In [123]:
prediction = prediction.drop(prediction.columns[1:-1],axis = 1)
prediction.to_csv('sub.csv', index=False)

In [43]:

submission.to_csv('sub.csv', index=False)

In [45]:
path = 'prediction/model1+two 0.17/'
model_list = os.listdir(path)

In [52]:
ins = pd.DataFrame()
ins['id']=train['id']
ins['true'] = train['is_iceberg']
for name in model_list:
    Cnnmodel.load_weights(os.path.join(path,name))
    pred = Cnnmodel.predict(X_train)
    ins[name] = np.squeeze(pred)

In [72]:
ins['gap'] = np.max(ins[ins.columns[2:]],axis = 1)-np.min(ins[ins.columns[2:]],axis = 1)

In [83]:
np.sum(((ins['gap']>0.2)*1))

451

In [84]:
(ins['gap']<0.2) and (np.all())

0       False
1       False
2        True
3       False
4       False
5       False
6       False
7        True
8       False
9       False
10       True
11      False
12       True
13       True
14      False
15      False
16       True
17      False
18       True
19       True
20      False
21       True
22      False
23      False
24      False
25      False
26       True
27      False
28       True
29      False
        ...  
1574     True
1575    False
1576    False
1577    False
1578    False
1579     True
1580    False
1581     True
1582     True
1583     True
1584    False
1585    False
1586     True
1587    False
1588    False
1589    False
1590    False
1591     True
1592    False
1593     True
1594    False
1595    False
1596    False
1597    False
1598    False
1599    False
1600    False
1601     True
1602     True
1603     True
Name: gap, Length: 1604, dtype: bool

In [102]:
np.sum(np.all(((ins[ins.columns[2:-1]]<0.5) & (ins[ins.columns[2:-1]]>0.1)),axis = 1)*1)

39

In [116]:
np.sum(np.all(prediction[prediction.columns[1:]]<0.2,axis = 1)*1)

4273